<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkBD0211ENSkillsNetwork885-2023-01-01"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="400"> </a>


<h1 align = "center"> Spark Fundamentals I - Introduction to Spark </h1>
<h2 align = "center"> Python - Working with RDD operations </h2>
<br align = "left">

**Related free online courses:**  

Related courses can be found in the following learning paths:

- [Spark Fundamentals path](http://cocl.us/Spark_Fundamentals_Path)
- [Big Data Fundamentals path](http://cocl.us/Big_Data_Fundamentals_Path)

<img src="http://spark.apache.org/images/spark-logo.png" height="100">


## Analyzing a log file

First let's download the tools that we need to use Spark in SN Labs.


In [1]:
!mamba install -qy -c conda-forge findspark pyspark

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext.getOrCreate()

23/08/18 06:34:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Run the following cells to ensure the __`LabData`__ folder is downloaded for this lab:


In [3]:
!pip install -Uq "skillsnetwork>=0.20.6"

In [4]:
import skillsnetwork
from packaging import version

if not hasattr(skillsnetwork, "__version__") or version.parse(skillsnetwork.__version__) < version.parse("0.20.6"):
    raise ValueError("Please install skillsnetwork>=0.20.6 or this lab won't work.")

In [5]:
await skillsnetwork.prepare("https://cocl.us/BD0211EN_Data", overwrite=True)

  0%|          | 0/26 [00:00<?, ?it/s]

Saved to '.'


In [6]:
from pathlib import Path
for path in Path("LabData").iterdir():
    print(path)

LabData/notebook.log
LabData/followers.txt
LabData/README.md
LabData/users.txt
LabData/nyctaxi.csv
LabData/nyctaxi100.csv
LabData/nycweather.csv
LabData/pom.xml
LabData/.DS_Store
LabData/nyctaxisub.csv
LabData/taxistreams.py


Now, let's create an RDD by loading the log file that we analyze in the Scala version of this lab.


In [7]:
logFile = sc.textFile("LabData/notebook.log")

### <span style="color: red">YOUR TURN:</span> 

#### In the cell below, filter out the lines that contains INFO


In [8]:
# WRITE YOUR CODE BELOW
info=logFile.filter(lambda line: "INFO" in line)
info.take(5)


['15/10/14 14:29:21 INFO SparkContext: Running Spark version 1.4.1',
 '15/10/14 14:29:22 INFO SecurityManager: Changing view acls to: notebook',
 '15/10/14 14:29:22 INFO SecurityManager: Changing modify acls to: notebook',
 '15/10/14 14:29:22 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(notebook); users with modify permissions: Set(notebook)',
 '15/10/14 14:29:23 INFO Slf4jLogger: Slf4jLogger started']

Double-click __here__ for the solution.
<!-- The correct answer is:
info = logFile.filter(lambda line: "INFO" in line)
-->


#### Count the lines:


In [9]:
# WRITE YOUR CODE BELOW

count_line=info.count()
print(count_line)

13438


Double-click __here__ for the solution.
<!-- The correct answer is:
info.count()
-->


#### Count the lines with "spark" in it by combining transformation and action.


In [10]:
# WRITE YOUR CODE BELOW

count_line_spark=info.filter(lambda line:'spark' in line).count()
print(count_line_spark)

156


Double-click __here__ for the solution.
<!-- The correct answer is:
info.filter(lambda line: "spark" in line).count()
-->


#### Fetch those lines as an array of Strings


In [11]:
# WRITE YOUR CODE BELOW

info.filter(lambda line:'spark' in line).collect()
info.filter(lambda line:'spark' in line).take(5)


['15/10/14 14:29:23 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkDriver@172.17.0.22:53333]',
 "15/10/14 14:29:23 INFO Utils: Successfully started service 'sparkDriver' on port 53333.",
 '15/10/14 14:29:23 INFO DiskBlockManager: Created local directory at /tmp/spark-fe150378-7bad-42b6-876b-d14e2c193eb6/blockmgr-c142f2f1-ebb6-4612-945b-0a67d156230a',
 '15/10/14 14:29:23 INFO HttpFileServer: HTTP File server directory is /tmp/spark-fe150378-7bad-42b6-876b-d14e2c193eb6/httpd-ed3f4ab0-7218-48bc-9d8a-3981b1cfe574',
 "15/10/14 14:29:24 INFO Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 35726."]

Double-click __here__ for the solution.
<!-- The correct answer is:
info.filter(lambda line: "spark" in line).collect()
-->


View the graph of an RDD using this command:


In [12]:
print(info.toDebugString())

b'(2) PythonRDD[7] at RDD at PythonRDD.scala:53 []\n |  LabData/notebook.log MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []\n |  LabData/notebook.log HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []'


## Joining RDDs

Next, you are going to create RDDs for the same README and the POM files that we used in the Scala version. 


In [13]:
readmeFile = sc.textFile("LabData/README.md")
pomFile = sc.textFile("LabData/pom.xml")

How many Spark keywords are in each file?


In [14]:
print(readmeFile.filter(lambda line: "Spark" in line).count())
print(pomFile.filter(lambda line: "Spark" in line).count())

18
2


Now do a WordCount on each RDD so that the results are (K,V) pairs of (word,count)


In [15]:
readmeCount = readmeFile.                    \
    flatMap(lambda line: line.split("   ")).   \
    map(lambda word: (word, 1)).             \
    reduceByKey(lambda a, b: a + b)
    
pomCount = pomFile.                          \
    flatMap(lambda line: line.split("   ")).   \
    map(lambda word: (word, 1)).            \
    reduceByKey(lambda a, b: a + b)

To see the array for either of them, just call the collect function on it.


In [16]:
print("Readme Count\n")
print(readmeCount.collect())

Readme Count



[Stage 7:>                                                          (0 + 2) / 2]

[('', 43), ('Spark is a fast and general cluster computing system for Big Data. It provides', 1), ('and Spark Streaming for stream processing.', 1), ('guide, on the [project web page](http://spark.apache.org/documentation.html)', 1), ('## Building Spark', 1), ('Spark is built using [Apache Maven](http://maven.apache.org/).', 1), (' build/mvn -DskipTests clean package', 1), ('Try the following command, which should return 1000:', 1), (' scala> sc.parallelize(1 to 1000).count()', 1), ('## Interactive Python Shell', 1), (' ./bin/pyspark', 1), ('And run the following command, which should also return 1000:', 1), ('Spark also comes with several sample programs in the `examples` directory.', 1), ('To run one of them, use `./bin/run-example <class> [params]`. For example:', 1), (' ./bin/run-example SparkPi', 1), ('will run the Pi example locally.', 1), ('You can set the MASTER environment variable when running examples to submit', 1), ('examples to a cluster. This can be a mesos:// or spark:/

In [17]:
print("Pom Count\n")
print(pomCount.collect())

Pom Count

[('<?xml version="1.0" encoding="UTF-8"?>', 1), ('  ~ Licensed to the Apache Software Foundation (ASF) under one or more', 1), ('  ~ contributor license agreements.  See the NOTICE file distributed with', 1), ('  ~ The ASF licenses this file to You under the Apache License, Version 2.0', 1), (' http://www.apache.org/licenses/LICENSE-2.0', 1), ('  ~ distributed under the License is distributed on an "AS IS" BASIS,', 1), ('  ~ limitations under the License.', 1), ('  -->', 1), ('', 841), ('<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">', 1), ('  <modelVersion>4.0.0</modelVersion>', 1), ('  <parent>', 1), (' <groupId>org.apache.spark</groupId>', 2), (' <artifactId>spark-parent_2.10</artifactId>', 1), (' <version>1.6.0-SNAPSHOT</version>', 1), ('  <properties>', 1), (' <sbt.project.name>examples</sbt.project.name>', 1), (' 

The join function combines the two datasets (K,V) and (K,W) together and get (K, (V,W)). Let's join these two counts together.


In [18]:
joined = readmeCount.join(pomCount)

Print the value to the console


In [19]:
joined.collect()

[('', (43, 841))]

Let's combine the values together to get the total count


In [30]:
joinedSum = joined.map(lambda k: (k[0], (k[1][0]+k[1][1])))

To check if it is correct, print the first five elements from the joined and the joinedSum RDD


In [31]:
print("Joined Individial\n")
print(joined.take(5))

print("\n\nJoined Sum\n")
print(joinedSum.take(5))

Joined Individial

[('', (43, 841))]


Joined Sum

[('', 884)]


## Shared variables

Normally, when a function passed to a Spark operation (such as map or reduce) is executed on a remote cluster node, it works on separate copies of all the variables used in the function. These variables are copied to each machine, and no updates to the variables on the remote machine are propagated back to the driver program. Supporting general, read-write shared variables across tasks would be inefficient. However, Spark does provide two limited types of shared variables for two common usage patterns: broadcast variables and accumulators.

### Broadcast variables

Broadcast variables are useful for when you have a large dataset that you want to use across all the worker nodes. A read-only variable is cached on each machine rather than shipping a copy of it with tasks. Spark actions are executed through a set of stages, separated by distributed “shuffle” operations. Spark automatically broadcasts the common data needed by tasks within each stage.


Read more here: [http://spark.apache.org/docs/latest/programming-guide.html#broadcast-variables](http://spark.apache.org/docs/latest/programming-guide.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkBD0211ENSkillsNetwork885-2023-01-01#broadcast-variables)

Create a broadcast variable. Type in:


In [32]:
broadcastVar = sc.broadcast([1,2,3])

To get the value, type in:


In [33]:
broadcastVar.value

[1, 2, 3]

### Accumulators

Accumulators are variables that can only be added through an associative operation. It is used to implement counters and sum efficiently in parallel. Spark natively supports numeric type accumulators and standard mutable collections. Programmers can extend these for new types. Only the driver can read the values of the accumulators. The workers can only invoke it to increment the value.

Create the accumulator variable. Type in:


In [34]:
accum = sc.accumulator(0)

Next parallelize an array of four integers and run it through a loop to add each integer value to the accumulator variable. Type in:


In [35]:
rdd = sc.parallelize([1,2,3,4])
def f(x):
    global accum
    accum += x

Next, iterate through each element of the rdd and apply the function f on it:


In [36]:
rdd.foreach(f)

To get the current value of the accumulator variable, type in:


In [37]:
accum.value

10

You should get a value of 10.

This command can only be invoked on the driver side. The worker nodes can only increment the accumulator.



## Key-value pairs

You have already seen a bit about key-value pairs in the Joining RDD section.

Create a key-value pair of two characters. Type in:


In [38]:
pair = ('a', 'b')

To access the value of the first index use [0] and [1] method for the 2nd.


In [39]:
print(pair[0])
print(pair[1])

a
b


<div class="alert alert-success alertsuccess" style="margin-top: 20px">
    <strong>Tip</strong>: Enjoyed using Jupyter notebooks with Spark? Get yourself a free 
    <a href="http://cocl.us/DSX_on_Cloud">IBM Cloud</a> account where you can use Data Science Experience notebooks
    and have <em>two</em> Spark executors for free!
</div>


### Summary
Having completed this exercise, you should now be able to describe Spark’s primary data abstraction, work with Resilient Distributed Dataset (RDD) operations, and utilize shared variables and key-value pairs.


This notebook is part of the free course on **cognitiveclass.ai** called *Spark Fundamentals I*. If you accessed this notebook outside the course, you can take this free self-paced course, online by going to: http://cocl.us/Spark_Fundamentals_I


### About the Authors:  
Hi! It's Alex Aklson, one of the authors of this notebook. I hope you found this lab educational! There is much more to learn about Spark but you are well on your way. Feel free to connect with me if you have any questions.
<hr>
